# Datos de los ligandos

## Obtención de los datos de los ligandos  cocristalizados con ERK2 desde Pubchem y Chembl

**Esta libreta tiene como objetivo extraer datos de las moléculas cocristalizadas con la proteína ERK2 a partir de las bases de datos CheMBL y PubChem**

> Puntos clave de esta libreta:
- Se extrae de la tabla de datos principal de la proteína las conformaciones que poseen a algún ligando en el sitio activo.
- Se crea un set de moléculas usando identificador PDB de cada molécula (cada molécula es designada con un id de 3 letras).
- Se usa el id de cada molécula (LIG) para obtener los datos de la página `http://www.rcsb.org/ligand/LIG`, correspondiente a cada ligando. Con ello obtenemos:
    - El smile isomérico de la molécula.
    - El InChI
    - El InChI Key
- Usamos el **InChI** para obtener los datos de cada molécula desde _**PUBCHEM**_ y _**CHEMBL**_.
- Guardamos los datos en un diccionario y a su vez lo exportamos a un archivo pinckle.

### Importamos las librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import glob, os, sys
# Añadimos la ruta del directorio raíz para tener acceso a los arvhivos y módulos
sys.path.append(r'..')

## Cargamos los datos de la tabla principal
Nombre de laproteína:

In [ ]:
# nombre de la proteína
prot_name = 'erk2'

In [ ]:
# Cargamos el dataframe principal de la proteína
path_string = ('../data/TABLA_MTDATA_ERK2_166_crys.csv')
path_to_json_file = glob.glob()